### READ ME

Use the code blocks below to answer each quiz question. Only print the output required for each question. Do not edit the comments at the top of each code cell. Otherwise, the auto-grader may misinterpret your results. See Question 0 as an an example of how to complete a task (leave it in your notebook; don't delete it):

In [6]:
# Question 0: Create a DataFrame with three rows and four columns. Name the 
# columns 'Col1', 'Col2', 'Col3', 'Col4'. Create an index for the DataFrame
# and give the rows the index values of 'Row1', 'Row2', 'Row3'. Place a value
# in each column equal to the {ColumnName/RowName}. e.g. Col1/Row1. Print
# the entire DataFrame.

import pandas as pd

df = pd.DataFrame(columns=['Col1', 'Col2', 'Col3', 'Col4'], index=['Row1', 'Row2', 'Row3'])

for col in df:
  for i, value in df[col].items():
    df.at[i, col] = f'{i}/{col}'

df

,Col1,Col2,Col3,Col4
Row1,Row1/Col1,Row1/Col2,Row1/Col3,Row1/Col4
Row2,Row2/Col1,Row2/Col2,Row2/Col3,Row2/Col4
Row3,Row3/Col1,Row3/Col2,Row3/Col3,Row3/Col4


In [30]:
# Question 1: 

# Write Python code using the requests and BeautifulSoup package to scrape http://quotes.toscrape.com/
# Begin by requesting the home page

import requests
url = 'http://quotes.toscrape.com/'
req = requests.get(url)
len(req.text)

11010

In [8]:
# Question 2:

# Create a session and login to the website using the following credentials:
#   username: [your first name]
#   password: checkpoint
# Print the csrf middleware token assigned during this login
from bs4 import BeautifulSoup

#set up the login page as the session
url_login = "http://quotes.toscrape.com/login"
session_requests = requests.session()
page_login = session_requests.get(url_login)

#turn the login page into a soup object
soup = BeautifulSoup(page_login.text, "html.parser")

#extract the csrf token from the login page
csrf = soup.findAll("input", {"name": "csrf_token"})[0]['value']

#set up the payload using credentials and csrf token
payload = {
    "username": "Jackson", 
    "password": "checkpoint", 
    "csrf_token": csrf
}

#login to the webpage
page_login = session_requests.post(
    url_login, 
    data = payload, 
    headers = dict(referer=url_login)
)

print('csrf: ' + csrf)
print(page_login.text)

csrf: QKVPtjkCeIGNhgdFZEULrmpbxWlnXAMwuqfoTYiSacHyRzBvDsOJ
<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="UTF-8">
	<title>Quotes to Scrape</title>
    <link rel="stylesheet" href="/static/bootstrap.min.css">
    <link rel="stylesheet" href="/static/main.css">
</head>
<body>
    <div class="container">
        <div class="row header-box">
            <div class="col-md-8">
                <h1>
                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>
                </h1>
            </div>
            <div class="col-md-4">
                <p>
                
                    <a href="/logout">Logout</a>
                
                </p>
            </div>
        </div>
    

<div class="row">
    <div class="col-md-8">

    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">
        <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our th

In [9]:
# Question 3:

# Print the headers dictionary from your login request
for k, v in page_login.headers.items():
  print(k + ': ' + v)

Server: nginx/1.17.7
Date: Fri, 17 Sep 2021 14:09:48 GMT
Content-Type: text/html; charset=utf-8
Transfer-Encoding: chunked
Connection: keep-alive
Vary: Accept-Encoding
Content-Encoding: gzip


In [27]:
# Question 4:

# Create a loop that will scrape the first 10 pages of quotes into a DataFrame
# Include the Quote, Author Name, Birthdate (see "about" page of each author), Birth Location, and Tags as columns
#   Since there are a variable number of tags, you can store all of them as a list object in the Tags column
# Sort the DataFrame by Birth Location descending
# Print the first five rows of the resulting DataFrame
import pandas as pd

df = pd.DataFrame(columns=['Quote', 'Author Name', 'Birthdate', 'Birth Location', 'Tags'])

def scrape_quotes():
  for outerCount in range(1, 11):
    req = requests.get(f'http://quotes.toscrape.com/page/{outerCount}/')
    soup_quote = BeautifulSoup(req.text, "html.parser")
    quotes = soup_quote.findAll('div', {'class': 'quote'})
    for innerCount in range(0, 10):
      #Find specific quote information
      quote_text = quotes[innerCount].find("span", {"class": "text"}).text
      author_name = quotes[innerCount].find("small", {"class": "author"}).text
      tags = []
      tags = quotes[innerCount].find("div", {"class": "tags"}).select("a")
      for iCount in range(len(tags)):
        tags[iCount] = tags[iCount].text
      
      #find specific Author Information
      author_page = quotes[innerCount].find("small", {"class": "author"}).findNextSibling("a").get("href")
      req = session_requests.get('http://quotes.toscrape.com' + author_page)
      soup_author = BeautifulSoup(req.text, "html.parser")
      birth_date = soup_author.find("span", {"class": "author-born-date"}).text
      birth_location = soup_author.find("span", {"class": "author-born-location"}).text.replace("in ", "")

      #put the scraped information into the dataframe
      arr = [quote_text, author_name, birth_date, birth_location, tags]
      iPlacement = innerCount + (outerCount - 1) * 10
      df.loc[iPlacement] = arr

scrape_quotes()
df.sort_values(by=['Birth Location'], ascending=False, inplace=True)
df.head(20)

,Quote,Author Name,Birthdate,Birth Location,Tags
65,"“Do not pity the dead, Harry. Pity the living,...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",[live-death-love]
31,"“Of course it is happening inside your head, H...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",[dumbledore]
90,"“The truth."" Dumbledore sighed. ""It is a beaut...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",[truth]
27,“It is impossible to live without failing at s...,J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",[]
48,"“It matters not what someone is born, but what...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",[dumbledore]
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...","[abilities, choices]"
11,“It takes a great deal of bravery to stand up ...,J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...","[courage, friends]"
82,"“Remember, if the time should come when you ha...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",[integrity]
34,"“To the well-organized mind, death is but the ...",J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...","[death, inspirational]"
9,"“A day without sunshine is like, you know, nig...",Steve Martin,"August 14, 1945","Waco, Texas, The United States","[humor, obvious, simile]"


In [28]:
# Question 5:

# Print the value_counts() of the Author Name column
print(df['Author Name'].value_counts())


Albert Einstein           10
J.K. Rowling               9
Marilyn Monroe             7
Mark Twain                 6
Dr. Seuss                  6
C.S. Lewis                 5
Jane Austen                5
Bob Marley                 3
Eleanor Roosevelt          2
Suzanne Collins            2
Ernest Hemingway           2
George R.R. Martin         2
Ralph Waldo Emerson        2
Mother Teresa              2
Charles Bukowski           2
J.M. Barrie                1
Pablo Neruda               1
Douglas Adams              1
Alfred Tennyson            1
Harper Lee                 1
Steve Martin               1
Jim Henson                 1
Friedrich Nietzsche        1
André Gide                 1
Jimi Hendrix               1
William Nicholson          1
Stephenie Meyer            1
George Bernard Shaw        1
E.E. Cummings              1
Alexandre Dumas fils       1
J.D. Salinger              1
Khaled Hosseini            1
Jorge Luis Borges          1
Terry Pratchett            1
Charles M. Sch